In [1]:
import os
os.chdir("../src/")
import pandas as pd
import s3fs
import import_inpi as inpi

In [3]:
pm, rep = inpi.import_inpi_s3()

/home/coder/work/data_inpi/src/import_inpi.py:109: DtypeWarning: Columns (7,8,10,11,16,30) have mixed types. Specify dtype option on import or set low_memory=False.
  personnes_morales = pd.read_csv(file_in, sep=";")
/home/coder/work/data_inpi/src/import_inpi.py:111: DtypeWarning: Columns (7,15,17,25,26,32,33,34,35,36,37,38,39,40,41,42,43,45,49,50) have mixed types. Specify dtype option on import or set low_memory=False.
  representants = pd.read_csv(file_in, sep=";")


In [4]:
pm.shape

(5001845, 31)

In [5]:
rep.shape

(11142886, 51)

In [137]:
import numpy as np
def clean_rep_inpi(df_rep):
    """_summary_Function that cleans data from Inpi

    Args:
        df_rep (_type_): _description_

    Returns:
        _type_: _description_
    """
    print(df_rep.shape)
    
    # nettoyage du nom des colonnes
    df_rep.columns = [c.lower() for c in df_rep.columns]
    df_rep = df_rep.rename({"code greffe": "code_greffe", 
                                "forme_juridique_x": "forme_juridique", 
                                "prénoms": "prenoms", "qualité": "qualite", 
                                "dénomination": "denomination",
                                "nationalité": "nationalite",
                                "id_représentant": "id_representant",
                                "siren.1": "siren_representant_pm"}, axis=1)
    # point sur les manquants
    print(df_rep.loc[(df_rep["nom_patronymique"].isna())|(df_rep["prenoms"].isna()),:].shape)

    # suppression de colonnes inutiles
    df_rep = df_rep.drop(['rep_perm_nom',
       'rep_perm_nom_usage', 'rep_perm_pseudo', 'rep_perm_prénoms',
       'rep_perm_date_naissance', 'rep_perm_ville_naissance',
       'rep_perm_pays_naissance', 'rep_perm_nationalité',
       'rep_perm_adr_ligne1', 'rep_perm_adr_ligne2', 'rep_perm_adr_ligne3',
       'rep_perm_code_postal', 'rep_perm_ville', 'rep_perm_code_commune',
       'rep_perm_pays', 'conjoint_collab_nom_patronym',
       'conjoint_collab_nom_usage', 'conjoint_collab_pseudo',
       'conjoint_collab_prénoms', 'conjoint_collab_date_fin'], axis=1)
    
    # Traitement des cas où le nom et les prénoms sont -
    df_rep.loc[df_rep["prenoms"] == "-", "prenoms"] = ""
    # on retire les termes en trop dans les noms
    re1 = "(INDIVISION SUCCESSORALE DE M\.\s|\sM\.\s|^M\.\s|MME\.\s|MR\.\s|MELLE\.\s|MONSIEUR\.?\s|MADAME\.?\s|MLLE\.\s|REPRESENTEE\sPAR\sMONSIEUR\.?\s|REPRESENTEE\sPAR\s|SUCCESSION\s|SUCCESSION\sDE\s)"
    # df_rep["top_civilite"] = df_rep["nom_patronymique"].str.contains(re1, regex=True)
    df_rep["nom_patronymique_cor"] = df_rep["nom_patronymique"]
    df_rep["nom_patronymique_cor"] = df_rep["nom_patronymique_cor"].str.replace(re1,"")
    
    # Traitement de la qualite
    df_rep["qualite_cor"] = (df_rep["qualite"].str.lower().str.strip())
    # nettoyage de la dénomination
    df_rep["denomination_cor"] = (df_rep["denomination"].str.lower())
    # Correction des adresses
    df_rep.loc[(df_rep["ville"].isna()) & (df_rep["adresse_ligne3"].isna() == False), "ville"] = \
    df_rep.loc[(df_rep["ville"].isna()) & (df_rep["adresse_ligne3"].isna() == False), "adresse_ligne3"]
    df_rep["commune"] = df_rep["ville"]
    df_rep["commune"] = df_rep["commune"].str.lower()
    df_rep["commune"] = (df_rep["commune"].str.replace("à", "a")
                                            .str.replace("è", "e")
                                            .str.replace("é", "e")
                                            .str.replace("ç", "c")
                                            .str.replace("ù", "u")
                                            .str.replace("ô", "o")
                                            .str.replace("saint", "st")
                                            .str.replace(" ", ""))

    # mettre date de naissance sous la forme AAAMMJJ
    df_rep["date_naissance_cor"] = df_rep["date_naissance"].str.replace("-","")

    # Traitement des particules
    df_rep["nom_patronymique_cor"] = (df_rep["nom_patronymique_cor"].str.replace(" DE LA ", "-DE-LA-")
        .str.replace(" DU ", "-DU-").str.replace(" LE ", "-LE-").str.replace(" LA ", "-LA-")
        .str.replace(" EL ", "-EL-")
        .str.replace(" SAINT ", "-SAINT-")
        .str.replace(" VAN ", "-VAN-")
        .str.replace(" DE ", "-DE-")
        .str.replace(" D ", "-D-")
        .str.replace("SAINT ", "SAINT-")
        #.str.replace("D ", "D-") -> induit des problemes
        .str.replace("DE ", "DE-")
        .str.replace("DU ", "DU-")
        .str.replace("LE ", "LE-")
        .str.replace("LA ", "LA-")
        .str.replace("EL ", "EL-")
        .str.replace("DI ", "DI-")
        .str.replace("BEN ", "BEN-")
        .str.replace("VAN ", "VAN-")
        .str.replace(" D'", "-D'"))
    # traitement des noms et prenoms
    df_rep["nom_cor"] = df_rep["nom_patronymique_cor"] 
    df_rep["prenom1_cor"] = df_rep["prenoms"] 
    df_rep["prenom2_cor"] = np.nan 
    df_rep["prenom3_cor"] = np.nan 
    # si prenom est vide on decoupe nom patronymique selon les espaces en mettant trois prenoms possibles
    temp = df_rep.loc[df_rep["prenoms"] == "", "nom_patronymique_cor"].str.split(pat=" ", n=3, expand=True)
    temp.columns = ["nom_cor", "prenom1_cor", "prenom2_cor", "prenom3_cor"]
    df_rep.loc[df_rep["prenoms"] == "", ["nom_cor", "prenom1_cor", "prenom2_cor", "prenom3_cor"]] = temp
    # quand nom et prenoms sont "" on les met à na
    df_rep.loc[df_rep["nom_cor"] == "", "nom_cor"] = np.nan
    df_rep.loc[df_rep["prenom1_cor"] == "", "prenom1_cor"] = np.nan
    # correction du type 
    df_rep["type"] = df_rep["type"].str.lower().str.replace(" ", "") 
    # correction des noms prenoms quand prenom est na, type est personne physique et le nom contient une virgule
    temp = df_rep.loc[(df_rep["prenoms"].isna()==True)&(df_rep["type"]=="p.physique")&
            (df_rep["nom_cor"].str.contains(",")),"nom_cor"].str.split(pat=" ", n=3, expand=True)
    temp.columns = ["nom_cor", "prenom1_cor", "prenom2_cor", "prenom3_cor"]
    temp["prenom1_cor"] = temp["prenom1_cor"].str.replace(",", "")
    temp["prenom2_cor"] = temp["prenom2_cor"].str.replace(",", "")
    temp["prenom3_cor"] = temp["prenom3_cor"].str.replace(",", "")
    df_rep.loc[(df_rep["prenoms"].isna()==True)&(df_rep["type"]=="p.physique")&
        (df_rep["nom_cor"].str.contains(",")),
        ["nom_cor", "prenom1_cor", "prenom2_cor", "prenom3_cor"]] = temp
    # traitement des cas X née Y, dans ce cas on separe X de Y et on place X en nom d'usage et Y en nom patronymique (nom_cor)
    temp = df_rep.loc[(df_rep["nom_cor"].str.contains("\sNEE\s|\snée\s|\sNée\s") == True)&
                        (df_rep["nom_usage"].isna()), "nom_cor"].str.split(pat="\sNEE\s|\snée\s|\sNée\s", n=1, expand=True)
    temp.columns = ["nom_usage_cor", "nom_cor"]
    df_rep.loc[(df_rep["nom_cor"].str.contains("\sNEE\s|\snée\s|\sNée\s") == True)&
                (df_rep["nom_usage"].isna()), ["nom_cor"]] = temp["nom_cor"]
    # filtre on ne conserve que les personnes physiques
    df_rep = df_rep.loc[df_rep["type"] == "p.physique",:]
    print(df_rep.shape)
    print(df_rep.loc[(df_rep["nom_cor"].isna())|(df_rep["prenom1_cor"].isna()),:].shape)
    return df_rep

In [138]:
df_rep = clean_rep_inpi(rep)

(11142886, 51)
(1119653, 51)


In [45]:
df_rep.columns

Index(['code_greffe', 'nom_greffe', 'numero_gestion', 'siren', 'type',
       'nom_patronymique', 'nom_usage', 'pseudonyme', 'prenoms',
       'denomination', 'siren_representant_pm', 'forme_juridique',
       'adresse_ligne1', 'adresse_ligne2', 'adresse_ligne3', 'code_postal',
       'ville', 'code_commune', 'pays', 'date_naissance', 'ville_naissance',
       'pays_naissance', 'nationalite', 'qualite', 'id_representant',
       'date_greffe', 'libelle_evt', 'source', 'year', 'file_path',
       'date_greffe', 'nom_patronymique_cor', 'top_particule', 'qualite_cor',
       'denomination_cor', 'commune', 'date_naissance_cor'],
      dtype='object')

In [69]:
df_rep.loc[df_rep["prenoms"] == "", ["nom_cor", "prenom1_cor", "prenom2_cor", "prenom3_cor"]].head()

,nom_cor,prenom1_cor,prenom2_cor,prenom3_cor
24689,SYED-ADIL,RASHID,None,None
25196,ISLAM,AWLAD,None,None
54431,ABD-EL-SALAM,ABD-EL-FATTAH,AHMED-RADWAN,None
70930,PIERRON,None,None,None
70931,PIERRON,None,None,None


In [72]:
df_rep.loc[:,["nom_cor", "prenom1_cor"]].head()

,nom_cor,prenom1_cor
0,BAZILE,Eric
1,BORFIGA,Philippe
2,NaN,NaN
3,GEORGHIOU,Jean-Christophe
4,NaN,NaN


In [79]:
df_rep.loc[(df_rep["prenoms"].isna()==True)&(df_rep["siren_representant_pm"].isna()==False), ].shape

(981443, 41)

In [86]:
df_rep.loc[(df_rep["prenoms"].isna()==True)&(df_rep["type"]=="p.physique"),:].shape # 8526

(8526, 41)

In [85]:
df_rep.loc[(df_rep["prenoms"].isna()==True)&(df_rep["type"]=="p.physique"),:].head()

,code_greffe,nom_greffe,numero_gestion,siren,type,nom_patronymique,nom_usage,pseudonyme,prenoms,denomination,...,nom_patronymique_cor,top_particule,qualite_cor,denomination_cor,commune,date_naissance_cor,nom_cor,prenom1_cor,prenom2_cor,prenom3_cor
674145,8903,Sens,2002D00078,775718463,p.physique,FEDERATION NATIONALE DE REVISION DES COOPERATI...,NaN,NaN,NaN,NaN,...,FEDERATION NATIONALE-DE-REVISION DES COOPERATI...,True,commissaire aux comptes suppléant,NaN,paris,NaN,FEDERATION NATIONALE-DE-REVISION DES COOPERATI...,NaN,NaN,NaN
683174,8903,Sens,1985D00013,331648063,p.physique,FEDERATION NATIONALE DE REVISION DES COOPERATI...,NaN,NaN,NaN,NaN,...,FEDERATION NATIONALE-DE-REVISION DES COOPERATI...,True,commissaire aux comptes suppléant,NaN,coin-les-cuvry,NaN,FEDERATION NATIONALE-DE-REVISION DES COOPERATI...,NaN,NaN,NaN
694286,8501,La Roche-sur-Yon,2007B01050,499451029,p.physique,ERNST & YOUNG ATLANTIQUE,NaN,NaN,NaN,NaN,...,ERNST & YOUNG ATLANTIQUE,False,commissaire aux comptes titulaire,NaN,nantescedex1,NaN,ERNST & YOUNG ATLANTIQUE,NaN,NaN,NaN
694299,8501,La Roche-sur-Yon,1998B00460,419107107,p.physique,FIDEA-FPL,NaN,NaN,NaN,NaN,...,FIDEA-FPL,False,commissaire aux comptes titulaire,NaN,st-gilles-croix-de-vie,NaN,FIDEA-FPL,NaN,NaN,NaN
694828,8501,La Roche-sur-Yon,1983B00115,327017992,p.physique,SELAS G/AUDIT,NaN,NaN,NaN,NaN,...,SELAS G/AUDIT,False,commissaire aux comptes titulaire,NaN,angers,NaN,SELAS G/AUDIT,NaN,NaN,NaN
